# Tutorial on OpenAI Function Calling

**Author:** John Adeojo  
**Company:** [Data-Centric Solutions](https://www.data-centric-solutions.com/)


In [1]:
import openai
import json

# Utility Function
Used to read the Open AI API from your apikey file.

In [2]:
import os 
import yaml
def read_config(path):
    """
    Reads API key from a configuration file.

    This function opens a configuration file named "apikeys.yml", reads the API key for OpenAI

    Returns:
    api_key (str): The API key for the Amadeus Flights API.
    """
    
    # Get the directory of the current script
    script_dir = path

    # Construct the full path to the configuration file
    file_path = os.path.join(script_dir, "apikeys.yml")

    with open(file_path, 'r') as stream:
        configs = yaml.safe_load(stream)
        API_KEY = configs['openai']['api_key']
            
    return API_KEY

In [3]:
path = r"C:\Users\johna\OneDrive\Documents\api_keys"  # Change to the location of your apikeys.yml
API_KEY = read_config(path)

# Setting up our functions
Let'smimagine we wanted to put a natural language interface on top of an API.
We can mimic an API with these timple functions. One to give us news headlines, and another to return a weather forecast.

In [4]:
import openai

def generate_response(topic, API_KEY):

    """
    Generates a random news headline for a given ticker
    """
    
    openai.api_key = API_KEY
    
    content = "You generate news headlines"
    prompt = f"generate a random news headline based on the f{topic}"
    
    response = openai.ChatCompletion.create(
        model="gpt-3.5-turbo-0613",
        messages=[
            {"role": "system", "content": content},
            {"role": "user", "content": prompt}
        ],
        n=1,
        stop=None,
        temperature=0.5,
    )

    return response.choices[0].message['content'].strip()


def get_current_news(topic):
    """Get the latest news on a given topic"""
    headline = generate_response(topic, API_KEY)
    
    news_round = {
        "topic": topic,
        "headline": headline
    }
    return json.dumps(news_round)

In [5]:
import pandas as pd
import numpy as np
def temp_model(date):
    """
    Converts a date to a number.

    Parameters:
    date (str): The date in 'YYYY-MM-DD' format.

    Returns:
    float: The number representation of the date.
    """
    random = np.random.uniform(1, 2)
    
    timestamp = pd.to_datetime(date).timestamp()
    number = np.round(np.log(timestamp) * np.log(random), 2)
    return number

def get_temperature(location, date):
    """
    Get the temperature for a given location, 
    at a given date and in a given 
    """

    temerature = temp_model(date)
    news_round = {
        "location": location,
        "date": date,
        "temerature": temerature,
        "scale": "Farenheit"
    }
    return json.dumps(news_round)

# Function Calling with One Function
Example of how function calling works with just one function

In [6]:
def function_calling_one_function(API_KEY, query):
    # Step 1: send the conversation and available functions to GPT
    functions = [
        {
            "name": "get_temperature",
            "description": "Get the temperature at a given location",
            "parameters": {
                "type": "object",
                "properties": {
                    "location": {
                        "type": "string",
                        "description": "Return the location cited in the query"
                    },
                    "date": {
                        "type": "string",
                        "description": "Return the date in in 'YYYY-MM-DD' format."
                    }
                 },
                },
                "required": ["location", "date"],
            }
    ]

    openai.api_key = API_KEY
    response = openai.ChatCompletion.create(
        model="gpt-3.5-turbo-0613",
        messages=[{"role": "system", "content": query}],
        functions=functions,
        function_call="auto",  # auto is default, but we'll be explicit
    )
    response_message = response["choices"][0]["message"]
    return response_message



In [48]:
import openai

def function_calling_multiple_function(API_KEY, query):
    # Step 1: send the conversation and available functions to GPT
    functions = [
        {
            "name": "get_current_news",
            "description": "Get the latest news headline for a given topic",
            "parameters": {
                "type": "object",
                "properties": {
                    "topic": {
                        "type": "string",
                        "description": "The news topic or topics required "
                    }
                 },
                },
                "required": ["topic"],
            },

        {
            "name": "get_temperature",
            "description": "Get the temperature at a given location",
            "parameters": {
                "type": "object",
                "properties": {
                    "location": {
                        "type": "string",
                        "description": "Return the location cited in the query"
                    },
                    "date": {
                        "type": "string",
                        "description": "Return the date in in 'YYYY-MM-DD' format."
                    }
                 },
                },
                "required": ["location", "date"],
            }
    ]

    openai.api_key = API_KEY
    response = openai.ChatCompletion.create(
        model="gpt-3.5-turbo-0613",
        messages=[{"role": "system", "content": query}],
        functions=functions,
        function_call="auto",  # auto is default, but we'll be explicit
    )
    response_message = response["choices"][0]["message"]
    return response_message

In [8]:
query = "What's the temperature forecast in berlin on the 1st of september 2023"
function_calling_one_function(API_KEY, query)

<OpenAIObject at 0x1fd8cdf6270> JSON: {
  "role": "assistant",
  "content": null,
  "function_call": {
    "name": "get_temperature",
    "arguments": "{\n  \"location\": \"Berlin\",\n  \"date\": \"2023-09-01\"\n}"
  }
}

In [54]:
query_1 = "What's the temperature forecast in berlin on the 1st of september 2023"
query_2 = "Give me the latest headline news on Berlin"
query_3 = "what's 10 + 10?"

In [41]:
query_1_response = function_calling_multiple_function(API_KEY, query_1)
query_1_response

<OpenAIObject at 0x1fd8ce1e6d0> JSON: {
  "role": "assistant",
  "content": null,
  "function_call": {
    "name": "get_temperature",
    "arguments": "{\n  \"location\": \"Berlin\",\n  \"date\": \"2023-09-01\"\n}"
  }
}

In [51]:
query_2_response = function_calling_multiple_function(API_KEY, query_2)
query_2_response

<OpenAIObject at 0x1fd8cdcf360> JSON: {
  "role": "assistant",
  "content": null,
  "function_call": {
    "name": "get_current_news",
    "arguments": "{\n  \"topic\": \"aliens in Berlin\"\n}"
  }
}

In [12]:
query_3 = function_calling_multiple_function(API_KEY, query_3)

# What happens if you try to do this without function calling?
Responses are hit and miss, sometimes doesn't respond to exact specifications. No additional flexibility.

In [66]:
def return_topic_args(query, API_KEY):

    openai.api_key = API_KEY
    
    content = "You parse the arguments defined in the prompt"
    prompt = f""" From the {query}, Return the news topic required """
    
    response = openai.ChatCompletion.create(
        model="gpt-3.5-turbo-0613",
        messages=[
            {"role": "system", "content": content},
            {"role": "user", "content": prompt}
        ],
        n=1,
        stop=None,
        temperature=0,
    )

    return response.choices[0].message['content'].strip()

In [67]:
return_topic_args(query_2, API_KEY)

'The news topic required is "Berlin".'

# Let's parse the function call outputs and feed them back into the Model to create a story

In [15]:
import json
def parse_function(response):
    parsed_output = response["function_call"]["arguments"]
    parsed_output = json.loads(parsed_output)
    return parsed_output

In [16]:
response = query_1_response
location = parse_function(response)['location']
date = parse_function(response)['date']

In [17]:
response = query_2_response
topic = parse_function(response)['topic']

In [18]:
news = get_current_news(topic)
news

'{"topic": "Berlin", "headline": "\\"Berlin Implements Innovative Green Initiative to Combat Climate Change\\""}'

In [19]:
data = get_temperature(location, date)
data

'{"location": "Berlin", "date": "2023-09-01", "temerature": 9.72, "scale": "Farenheit"}'

In [20]:
def create_story(data, news, API_KEY):

    """
    Generates a news story for a given some weather data and a news headline
    """
    
    openai.api_key = API_KEY
    
    content = "You are a journalist writing about how climate is impact current events."
    prompt = f"""write a short pargraph about the {data}, given {news}"""
    
    response = openai.ChatCompletion.create(
        model="gpt-3.5-turbo-0613",
        messages=[
            {"role": "system", "content": content},
            {"role": "user", "content": prompt}
        ],
        n=1,
        stop=None,
        temperature=0.5,
    )

    return response.choices[0].message['content'].strip()

In [21]:
create_story(data, news, API_KEY)

"Berlin, known for its rich history and vibrant culture, is taking a bold step towards combating climate change. The city has recently implemented an innovative green initiative, as reported on September 1, 2023. With temperatures reaching 9.72 degrees Fahrenheit, Berlin is feeling the impact of global warming firsthand. However, the city's proactive approach to sustainability aims to mitigate these effects. This new initiative is not only a testament to Berlin's commitment to environmental stewardship but also serves as an inspiration for other cities worldwide to take similar actions in the fight against climate change."